In [4]:
import os
from collections import namedtuple
import numpy as np
import pandas as pd
import ifcopenshell
import ifcopenshell.util.placement
import ifcopenshell.util.classification
from pathlib import Path
import ifcopenshell.geom
import ifcopenshell.util.element as element
import ifcopenshell.guid
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ifcopenshell.util.shape as shape_util

settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

In [6]:
model_path = "C:\\Users\\simon\\OneDrive\\Desktop\\School\\41934 - Advanced BIM\\Models"
str_model_name = "Precast-STR.ifc"


str_model_path = Path(model_path) / str_model_name
print(f"File path for structural model: {str_model_path}")

# Open the IFC model
str_model = ifcopenshell.open(str(str_model_path))
ModelHasColor = True
print(f"Model schema structural model: {str_model.schema}")

File path for structural model: C:\Users\simon\OneDrive\Desktop\School\41934 - Advanced BIM\Models\Precast-STR.ifc
Model schema structural model: IFC2X3
Model schema structural model: IFC2X3


In [7]:
structural_types = [
    "IfcBeam",
    "IfcColumn",
    "IfcSlab",
    "IfcMember",
    "IfcPlate",
    "IfcWall",
    "IfcWallStandardCase",  
]

# Dictionary: { "IfcBeam": [elements], "IfcColumn": [...], ... }
elements_by_type = {
    t: str_model.by_type(t)
    for t in structural_types
}

# Print counts
for t, elems in elements_by_type.items():
    print(f"{t}: {len(elems)} elements")


IfcBeam: 510 elements
IfcColumn: 121 elements
IfcSlab: 1 elements
IfcMember: 0 elements
IfcPlate: 0 elements
IfcWall: 15 elements
IfcWallStandardCase: 0 elements


In [ ]:
def get_storey_z_ranges(model: ifcopenshell.file):
    storeys = model.by_type("IfcBuildingStorey")
    storeys_sorted = sorted(storeys, key=lambda s: getattr(s, "Elevation", 0.0) or 0.0)
    ranges = {}
    for i, st in enumerate(storeys_sorted):
        z0 = float(getattr(st, "Elevation", 0.0) or 0.0)
        if i < len(storeys_sorted) - 1:
            z1 = float(getattr(storeys_sorted[i + 1], "Elevation", z0 + 10000.0) or (z0 + 10000.0))
        else:
            z1 = z0 + 10000.0
        ranges[st.GlobalId] = (z0, z1, st.Name)
    return ranges


def detect_element_floor(bmin, bmax, storey_z):
    zmin, zmax = float(bmin[2]), float(bmax[2])
    for st_gid, (zs, ze, _) in storey_z.items():
        if not (zmax < zs or zmin > ze):
            return st_gid
    return None


def assigned_storey_guid(element, model: ifcopenshell.file | None = None):
    rels = getattr(element, "ContainedInStructure", None) or []
    for rel in rels:
        related = getattr(rel, "RelatingStructure", None)
        if related is None:
            continue
        if related.is_a("IfcBuildingStorey"):
            return related.GlobalId
        if related.is_a("IfcSpace") and model is not None:
            space = related
            space_rels = getattr(space, "ContainedInStructure", None) or []
            for r2 in space_rels:
                rs = getattr(r2, "RelatingStructure", None)
                if rs is not None and rs.is_a("IfcBuildingStorey"):
                    return rs.GlobalId
    return None


def list_storeys(model):
    storeys = model.by_type("IfcBuildingStorey")
    storeys_sorted = sorted(storeys, key=lambda s: float(getattr(s, 'Elevation', 0.0) or 0.0))
    return [(st.GlobalId, st.Name or '<unnamed>', float(getattr(st, 'Elevation', 0.0) or 0.0)) for st in storeys_sorted]

In [ ]:
import plotly.graph_objects as go

# --------------------------------------------------------
# 3D BOX WIREFRAME BUILDER
# --------------------------------------------------------
def box_mesh(min_corner, max_corner):
    """Return wireframe vertices for a bounding box."""
    x0, y0, z0 = min_corner
    x1, y1, z1 = max_corner

    pts = np.array([
        [x0,y0,z0], [x1,y0,z0], [x1,y1,z0], [x0,y1,z0],
        [x0,y0,z1], [x1,y0,z1], [x1,y1,z1], [x0,y1,z1]
    ])

    edges = [
        (0,1),(1,2),(2,3),(3,0),     # bottom
        (4,5),(5,6),(6,7),(7,4),     # top
        (0,4),(1,5),(2,6),(3,7)      # vertical edges
    ]

    xs = []; ys = []; zs = []

    for i,j in edges:
        xs += [pts[i,0], pts[j,0], None]
        ys += [pts[i,1], pts[j,1], None]
        zs += [pts[i,2], pts[j,2], None]

    return xs, ys, zs


# --------------------------------------------------------
# BUILD PLOT
# --------------------------------------------------------
fig = go.Figure()

# ✅ Beams → red
for guid, (mn, mx) in beam_boxes.items():
    xs, ys, zs = box_mesh(mn, mx)
    fig.add_trace(go.Scatter3d(
        x=xs, y=ys, z=zs, mode="lines",
        line=dict(color="red", width=4),
        name=f"Beam {guid[:6]}"
    ))

# ✅ Slabs → blue
for guid, (mn, mx) in slab_boxes.items():
    xs, ys, zs = box_mesh(mn, mx)
    fig.add_trace(go.Scatter3d(
        x=xs, y=ys, z=zs, mode="lines",
        line=dict(color="blue", width=4),
        name=f"Slab {guid[:6]}"
    ))

# ✅ Columns → green
for guid, (mn, mx) in column_boxes.items():
    xs, ys, zs = box_mesh(mn, mx)
    fig.add_trace(go.Scatter3d(
        x=xs, y=ys, z=zs, mode="lines",
        line=dict(color="green", width=4),
        name=f"Column {guid[:6]}"
    ))

# --------------------------------------------------------
# FINAL PLOT SETTINGS
# --------------------------------------------------------
fig.update_layout(
    title="Beams (red), Slabs (blue), Columns (green)",
    width=1200, height=800,
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z",
        aspectmode="data",
    ),
    showlegend=True
)

fig.show()



In [10]:
# Floor utilities from analysis.py
from analysis import (
    get_storey_z_ranges,
    detect_element_floor,
    assigned_storey_guid as assigned_storey_guid_mod,
    get_element_bbox,
    list_storeys,
 )

# Compute storey ranges and display a simple list
storey_z = get_storey_z_ranges(str_model)
print('Storeys (GUID, Name, Elevation):')
for gid, name, elev in list_storeys(str_model):
    print(f'- {name} ({gid}): {elev:.3f}')

Storeys (GUID, Name, Elevation):
- L-1 (1ztVFtqpvDmvN1KPj_gIa1): -4000.000
- L0 (2JF4e6axWHqu3u0C1FZlmi): 0.000
- L-00 (15NoqEhN5D$QPHoOKwEx61): 0.000
- L01 (1ZGO8hrFCHqw0v0026FpFv): 4000.000
- L-01 (15BwyDTUT0XBsu0w5XsGFg): 4000.000
- L2 (1ztVFtqpvDmvN1KPj_gIds): 8000.000
- L3 (1ztVFtqpvDmvN1KPj_gIdf): 12000.000
- L4 (1ztVFtqpvDmvN1KPj_gIdc): 16000.000


In [11]:
# Helper functions to compare assigned vs geometry-derived storey
def pretty_storey(gid: str) -> str:
    if gid is None:
        return 'None'
    info = storey_z.get(gid)
    if info:
        zs, ze, name = info
        return f"{name or '<unnamed>'} ({gid})"
    return gid

def check_element_floor(e):
    bbox = get_element_bbox(e)
    bmin, bmax = bbox['min'], bbox['max']
    assigned_local = assigned_storey_guid(e, model=str_model)  # from this notebook
    assigned_mod = assigned_storey_guid_mod(e, model=str_model)  # from analysis.py
    actual = detect_element_floor(bmin, bmax, storey_z)
    print(f"{e.is_a()} {e.GlobalId}:")
    print(f"  assigned (notebook): {pretty_storey(assigned_local)}")
    print(f"  assigned (analysis): {pretty_storey(assigned_mod)}")
    print(f"  actual by geometry:  {pretty_storey(actual)}")
    print(f"  z-min/max: ({bmin[2]:.3f}, {bmax[2]:.3f})")

In [12]:
# Run checks on a small sample from each type
types = ['IfcBeam','IfcSlab','IfcColumn','IfcWall','IfcWallStandardCase']
for t in types:
    elems = list(str_model.by_type(t))[:3]
    if not elems:
        continue
    print(f"\n=== Testing {t} (first {len(elems)}) ===")
    for e in elems:
        check_element_floor(e)


=== Testing IfcBeam (first 3) ===
IfcBeam 1PbTsAK$P39xW3_P0T0tln:
  assigned (notebook): L0 (2JF4e6axWHqu3u0C1FZlmi)
  assigned (analysis): L0 (2JF4e6axWHqu3u0C1FZlmi)
  actual by geometry:  L-1 (1ztVFtqpvDmvN1KPj_gIa1)
  z-min/max: (-0.420, 0.000)
IfcBeam 1PbTsAK$P39xW3_P0T0sLt:
  assigned (notebook): L0 (2JF4e6axWHqu3u0C1FZlmi)
  assigned (analysis): L0 (2JF4e6axWHqu3u0C1FZlmi)
  actual by geometry:  L-1 (1ztVFtqpvDmvN1KPj_gIa1)
  z-min/max: (-0.420, 0.000)
IfcBeam 1PbTsAK$P39xW3_P0T0sLT:
  assigned (notebook): L0 (2JF4e6axWHqu3u0C1FZlmi)
  assigned (analysis): L0 (2JF4e6axWHqu3u0C1FZlmi)
  actual by geometry:  L-1 (1ztVFtqpvDmvN1KPj_gIa1)
  z-min/max: (-0.420, 0.000)

=== Testing IfcSlab (first 1) ===
IfcSlab 1ztVFtqpvDmvN1KPj_gIbJ:
  assigned (notebook): L-1 (1ztVFtqpvDmvN1KPj_gIa1)
  assigned (analysis): L-1 (1ztVFtqpvDmvN1KPj_gIa1)
  actual by geometry:  L-1 (1ztVFtqpvDmvN1KPj_gIa1)
  z-min/max: (-4.300, -4.000)

=== Testing IfcColumn (first 3) ===
IfcColumn 2lEpoWvzb0Bx$So4Cxnr